In [ ]:
# 安裝必要套件
!pip install langgraph openai


In [ ]:
# 導入模組
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from openai import OpenAI
from typing import Dict
from google.colab import userdata

# 定義狀態結構
@dataclass
class SummaryState:
    input: str  # 輸入新聞
    summary: str = ""  # 生成的摘要
    post: str = ""  # 生成的X貼文
    summary_feedback: str = ""  # 摘要反饋
    style_feedback: str = ""  # 風格反饋
    iteration: int = 0  # 迭代次數

# 初始化OpenAI客戶端
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 從Colab環境獲取金鑰

# 語言模型調用函數（增加錯誤處理）
def llm_call(prompt: str, max_tokens: int = 150) -> str:
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=0.5
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"錯誤：{str(e)}"

# 摘要生成節點
def summary_node(state: SummaryState) -> Dict:
    prompt = f"將以下1000字新聞摘要為100字，保留主要事件、影響，語氣中立，語句流暢，嚴格控制字數：\n{state.input}"
    state.summary = llm_call(prompt, max_tokens=120)
    return {"summary": state.summary}

# 摘要檢查節點
def summary_check_node(state: SummaryState) -> Dict:
    prompt = f"檢查以下摘要是否包含新聞核心事件與影響，語句是否流暢，字數是否約100字。若有問題，提供建議；若無，回覆‘無問題’：\n摘要：{state.summary}\n新聞：{state.input}"
    state.summary_feedback = llm_call(prompt, max_tokens=100)
    state.iteration += 1
    return {"summary_feedback": state.summary_feedback, "iteration": state.iteration}

# 貼文生成節點
def post_node(state: SummaryState) -> Dict:
    prompt = f"將以下100字摘要改編為50~100字X貼文，語氣中立，簡潔吸睛，包含2~3個標籤（如#科技），嚴格控制字數：\n{state.summary}"
    state.post = llm_call(prompt, max_tokens=80)
    return {"post": state.post}

# 風格檢查節點
def style_node(state: SummaryState) -> Dict:
    prompt = f"檢查以下X貼文是否語氣中立、標籤適切（2~3個，符合主題）。若有問題，提供建議；若無，回覆‘無問題’：\n貼文：{state.post}"
    state.style_feedback = llm_call(prompt, max_tokens=100)
    return {"style_feedback": state.style_feedback}

# 構建工作流程
workflow = StateGraph(SummaryState)
workflow.add_node("summary_node", summary_node)
workflow.add_node("summary_check_node", summary_check_node)
workflow.add_node("post_node", post_node)
workflow.add_node("style_node", style_node)
workflow.set_entry_point("summary_node")
workflow.add_edge("summary_node", "summary_check_node")
workflow.add_conditional_edges(
    "summary_check_node",
    lambda state: "post_node" if "無問題" in state.summary_feedback or state.iteration >= 2 else "summary_node",
    {"summary_node": "summary_node", "post_node": "post_node"}
)
workflow.add_edge("post_node", "style_node")
workflow.add_edge("style_node", END)
graph = workflow.compile()

# 範例新聞（1000字，實際使用可替換）
news = """
人工智慧新突破：醫療影像診斷提升90%準確率
人工智慧（AI）在醫療領域再創里程碑！一項由國際研究團隊開發的新技術，將醫療影像診斷的準確率提升至90%，遠超傳統方法的70%。這項技術利用深度學習模型，結合大數據分析，能自動識別醫療影像中的異常區域，並生成診斷報告。研究顯示，這不僅大幅提升診斷效率，還能將癌症篩查的誤診率降低30%，為患者提供更早的治療機會。團隊表示，這項技術可減輕醫生負擔，讓他們專注於複雜病例的處理。專家認為，AI在醫療影像診斷的應用前景廣闊，未來有望整合至醫院系統，推動智慧醫療發展。然而，技術普及仍需解決資料隱私與模型透明度等挑戰。這項突破已在多個醫學會議上獲得關注，預計將影響全球醫療產業。
[假設此處還有約800字的擴充內容，描述技術細節、案例與未來展望等]
"""

# 執行整合系統
result = graph.invoke(SummaryState(input=news))
print(f"最終摘要：\n{result['summary']}")
print(f"最終X貼文：\n{result['post']}")
print(f"摘要反饋：\n{result['summary_feedback']}")
print(f"風格反饋：\n{result['style_feedback']}")